In [4]:
# Phase 3: Weekly Lineup Optimization Setup
# Create this in a new notebook: 03_weekly_optimization.ipynb

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.optimize import linprog
import warnings
warnings.filterwarnings('ignore')

print("PHASE 3: WEEKLY LINEUP OPTIMIZATION")
print("=" * 50)

# Load the dataset
df = pd.read_pickle('../data/fantasy_data_2023.pkl')

# Create metadata
metadata = {
    'positions': sorted(df['position'].unique().tolist()),
    'weeks_covered': sorted(df['week'].unique().tolist()),
    'total_records': len(df),
    'unique_players': df['player_display_name'].nunique()
}

print(f"Dataset loaded: {df.shape[0]:,} records")
print(f"Positions: {metadata['positions']}")
print(f"Weeks: {len(metadata['weeks_covered'])} weeks")

# Your league lineup requirements (10 starters)
lineup_requirements = {
    'QB': 1,
    'RB': 2,
    'WR': 2,
    'TE': 1,
    'FLEX1': 1,  # RB/WR/TE
    'FLEX2': 1,  # RB/WR/TE
    'K': 1,
    'DEF': 1
}

print(f"\nYour league lineup requirements: {lineup_requirements}")
print(f"Total starters: {sum(lineup_requirements.values())}")

# Define flex eligibility
flex_eligible_positions = ['RB', 'WR', 'TE']
print(f"FLEX eligible positions: {flex_eligible_positions}")

# potentially start:
print(f"\nPossible lineup combinations:")
print(f"- Max RBs: 4 (2 RB + 2 FLEX)")
print(f"- Max WRs: 4 (2 WR + 2 FLEX)")  
print(f"- Max TEs: 3 (1 TE + 2 FLEX)")
print(f"- Always: 1 QB, 1 K, 1 DEF")


PHASE 3: WEEKLY LINEUP OPTIMIZATION
Dataset loaded: 6,670 records
Positions: ['DEF', 'K', 'QB', 'RB', 'TE', 'WR']
Weeks: 22 weeks

Your league lineup requirements: {'QB': 1, 'RB': 2, 'WR': 2, 'TE': 1, 'FLEX1': 1, 'FLEX2': 1, 'K': 1, 'DEF': 1}
Total starters: 10
FLEX eligible positions: ['RB', 'WR', 'TE']

Possible lineup combinations:
- Max RBs: 4 (2 RB + 2 FLEX)
- Max WRs: 4 (2 WR + 2 FLEX)
- Max TEs: 3 (1 TE + 2 FLEX)
- Always: 1 QB, 1 K, 1 DEF
